#Modelagem (Gold - Modelo Estrela)

Dimensões e tabela fato a partir das tabelas Silver.


### Dimensão de Business

In [0]:
from pyspark.sql import functions as F

df_business = spark.table("silver_yelp_business")
gold_dim_business = (
    df_business.select(
        "business_id",
        "name",
        "city",
        "state",
        "postal_code",
        "latitude",
        "longitude",
        "categories",
        "stars",
        "review_count",
        "is_open")
)
(gold_dim_business
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold_dim_business"))

### Fato de Reviews

In [0]:
df_review = spark.table("silver_yelp_review")

gold_fact_review = (
    df_review.select(
        "review_id",
        "business_id",
        "user_id",
        "review_stars",
        "review_ts",
        "review_date",
        "useful",
        "funny",
        "cool")
)
(gold_fact_review
 .write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("gold_fact_review"))

### Validação

In [0]:
spark.sql("SELECT COUNT(*) AS n FROM gold_dim_business").show()
spark.sql("SELECT COUNT(*) AS n FROM gold_fact_review").show()

spark.sql("""SELECT MIN(review_stars) AS min_stars, MAX(review_stars) AS max_stars
FROM gold_fact_review""").show()

+------+
|     n|
+------+
|150346|
+------+

+-------+
|      n|
+-------+
|3000000|
+-------+

+---------+---------+
|min_stars|max_stars|
+---------+---------+
|      1.0|      5.0|
+---------+---------+

